# Stupid word generation with markov chains

**WHY?**
Because I can and most of LLMs are not funny enough

In [1]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/short-jokes/shortjokes.csv


In [2]:
!pip install nltk

In [3]:
import pandas as pd
import numpy as np
import nltk
import random
from nltk import bigrams, trigrams
from nltk.corpus import stopwords
from collections import defaultdict, Counter

In [4]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
df = pd.read_csv('/kaggle/input/short-jokes/shortjokes.csv')
tokenized_jokes = [nltk.word_tokenize(jokes.lower()) for jokes in df['Joke']]

In [10]:
def build_transition_matrix(texts):
    transition_matrix = defaultdict(Counter)
    for sentence in texts:
        for word1, word2 in zip(sentence, sentence[1:]):
            transition_matrix[word1][word2] += 1
    for word1 in transition_matrix:
        total = sum(transition_matrix[word1].values())
        for word2 in transition_matrix[word1]:
            transition_matrix[word1][word2] /= total
    return transition_matrix


In [11]:
# Build transition matrix for jokes
joke_matrix = build_transition_matrix(tokenized_jokes)

In [13]:
def generate_text(start_word, transition_matrix, length=15):
    current_word = start_word
    generated_text = [current_word]
    
    for _ in range(length - 1):
        next_word_probs = transition_matrix.get(current_word, {})
        if not next_word_probs:
            break
        next_word = random.choices(list(next_word_probs.keys()), weights=next_word_probs.values())[0]
        generated_text.append(next_word)
        current_word = next_word

    return ' '.join(generated_text)

In [12]:
def generate_joke_from_prompt(prompt):
    prompt_tokens = nltk.word_tokenize(prompt.lower())
    start_word = prompt_tokens[0]  # Use the first word of the prompt as the start of the joke
    joke = generate_text(start_word, joke_matrix, length=20)
    
    return joke.capitalize()

In [61]:
# Example usage: Get a joke based on a user prompt
user_prompt = "Sadness"
generated_joke = generate_joke_from_prompt(user_prompt)
print(generated_joke)

Sadness to hundreds and one . whenever i 'll do porn stars ... that sticks to have no one torta


In [62]:
import pickle

with open('markov_chain_model.pkl', 'wb') as f:
    pickle.dump(joke_matrix, f)

Markov Chain model saved.
